In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer

# Load datasets
static_df = pd.read_csv("/Users/isha/Datathon/venv/static_client_data.csv")
time_series_df = pd.read_csv("/Users/isha/Datathon/adjusted_timeseries_SCN_02.csv")


# Convert preferred_asset_classes to lists (if stored as strings)
static_df["preferred_asset_classes"] = static_df["preferred_asset_classes"].apply(lambda x: eval(x) if isinstance(x, str) else x)

# One-Hot Encode categorical columns
one_hot_cols = ["gender", "employment_status", "investment_goals"]
one_hot_encoder = OneHotEncoder(drop="first", sparse_output=False)
one_hot_encoded = one_hot_encoder.fit_transform(static_df[one_hot_cols])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols))

# Label Encode risk_appetite
label_encoder = LabelEncoder()
static_df["risk_appetite_encoded"] = label_encoder.fit_transform(static_df["risk_appetite"])

# Multi-Hot Encode preferred_asset_classes
mlb = MultiLabelBinarizer()
multi_hot_encoded = mlb.fit_transform(static_df["preferred_asset_classes"])
multi_hot_df = pd.DataFrame(multi_hot_encoded, columns=[f"asset_{cls}" for cls in mlb.classes_])

# Drop original categorical columns and concatenate encoded features
static_df.drop(columns=one_hot_cols + ["risk_appetite", "preferred_asset_classes"], inplace=True)
static_df = pd.concat([static_df, one_hot_df, multi_hot_df], axis=1)

# Merge with time-series data (preserving all time-series rows)
merged_df = time_series_df.merge(static_df, on="client_id", how="left")




# Save processed dataset
merged_df.to_csv("/Users/isha/Datathon/static_time_series_ps3.csv", index=False)

print("✅ Encoding & Merging complete! Merged dataset shape:", merged_df.shape)


✅ Encoding & Merging complete! Merged dataset shape: (36000, 31)


In [39]:
time_series_df.shape


(36000, 9)

Check stationarity for each client and feature

In [40]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

df = pd.read_csv("/Users/isha/Datathon/static_time_series_ps3.csv")

time_features = [
    'portfolio_value', 
    'equity_allocation_pct', 
    'fixed_income_allocation_pct', 
    'monthly_contribution', 
    'market_volatility_index', 
    'macroeconomic_score', 
    'sentiment_index'
]

# Initialize an empty list to store the results
all_results = []

# Check stationarity for each client and feature
for client_id in df['client_id'].unique():
    client_data = df[df['client_id'] == client_id]
    
    for feature in time_features:
        # Extract the time series for the current feature
        series = client_data[feature].values
        
        # Run ADF test
        result = adfuller(series, autolag='AIC')
        
        # Store results in the list
        all_results.append({
            'client_id': client_id,
            'feature': feature,
            'adf_statistic': result[0],
            'p_value': result[1],
            'is_stationary': result[1] < 0.05  # p-value < 0.05 → stationarity
        })

# Create the DataFrame after the loop
stationarity_results = pd.DataFrame(all_results)

# Save or display results
print(stationarity_results)


                                 client_id                      feature  \
0     2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2              portfolio_value   
1     2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2        equity_allocation_pct   
2     2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2  fixed_income_allocation_pct   
3     2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2         monthly_contribution   
4     2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2      market_volatility_index   
...                                    ...                          ...   
6995  41b1bc9d-b7ec-4092-803b-6ae2a1023605  fixed_income_allocation_pct   
6996  41b1bc9d-b7ec-4092-803b-6ae2a1023605         monthly_contribution   
6997  41b1bc9d-b7ec-4092-803b-6ae2a1023605      market_volatility_index   
6998  41b1bc9d-b7ec-4092-803b-6ae2a1023605          macroeconomic_score   
6999  41b1bc9d-b7ec-4092-803b-6ae2a1023605              sentiment_index   

      adf_statistic       p_value  is_stationary  
0         -1.729305  4.160802e-01          False

Apply differencing if non-stationary

In [34]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

# Load data
df = pd.read_csv("/Users/isha/Datathon/static_time_series_ps3.csv")

# List of time-series features to check
time_features = [
    'portfolio_value', 
    'equity_allocation_pct', 
    'fixed_income_allocation_pct', 
    'monthly_contribution', 
    'market_volatility_index', 
    'macroeconomic_score', 
    'sentiment_index'
]

# List to store transformed data
differenced_data = []

# Loop over each client
for client_id in df['client_id'].unique():
    client_data = df[df['client_id'] == client_id].copy()

    for feature in time_features:
        series = client_data[feature].values

        # ADF test
        try:
            p_value = adfuller(series, autolag='AIC')[1]
        except:
            p_value = 1  # Treat as non-stationary if test fails

        # Apply differencing if non-stationary
        if p_value >= 0.05:
            diff_series = np.diff(series)
            diff_series = np.insert(diff_series, 0, 0)  # Pad to maintain length
        else:
            diff_series = series

        # Add new stationary feature column
        client_data[feature + '_stationary'] = diff_series

    # Store updated client data
    differenced_data.append(client_data)

# Combine everything back
df_stationary = pd.concat(differenced_data, ignore_index=True)

# Save to file
df_stationary.to_csv("/Users/isha/Datathon/stationary_data1.csv", index=False)
print("✅ Stationarity transformation applied to all features and saved.")


✅ Stationarity transformation applied to all features and saved.


In [16]:
df_stationary

,client_id,month,portfolio_value,equity_allocation_pct,fixed_income_allocation_pct,monthly_contribution,market_volatility_index,macroeconomic_score,sentiment_index,age,...,asset_Mutual Funds,asset_Real Estate,asset_Stocks,portfolio_value_stationary,equity_allocation_pct_stationary,fixed_income_allocation_pct_stationary,monthly_contribution_stationary,market_volatility_index_stationary,macroeconomic_score_stationary,sentiment_index_stationary
0,2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2,01-03-2022,53094.36,63.89460,32.72620,1188.90,17.16,3.83,7.25,53,...,0,1,0,0.00,63.89460,32.72620,1188.90,17.16,3.83,7.25
1,2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2,01-04-2022,54580.94,57.82784,39.74048,1738.97,12.01,2.56,3.82,53,...,0,1,0,1486.58,57.82784,39.74048,1738.97,12.01,2.56,3.82
2,2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2,01-05-2022,56255.62,37.69136,63.02192,1175.53,24.98,4.62,6.21,53,...,0,1,0,1674.68,37.69136,63.02192,1175.53,24.98,4.62,6.21
3,2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2,01-06-2022,58008.47,44.51416,55.13352,1968.66,22.68,1.99,6.49,53,...,0,1,0,1752.85,44.51416,55.13352,1968.66,22.68,1.99,6.49
4,2eef3ba0-a7c7-42b1-a3ac-3c33aedcb5f2,01-07-2022,59831.30,36.42822,64.48234,1529.49,11.39,2.23,7.39,53,...,0,1,0,1822.83,36.42822,64.48234,1529.49,11.39,2.23,7.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,41b1bc9d-b7ec-4092-803b-6ae2a1023605,01-10-2024,56962.78,19.08540,84.53380,1862.76,20.85,1.45,5.99,64,...,0,1,0,1884.81,19.08540,84.53380,1862.76,20.85,1.45,5.99
35996,41b1bc9d-b7ec-4092-803b-6ae2a1023605,01-11-2024,59890.52,43.59216,56.19952,1926.25,22.79,3.21,5.02,64,...,0,1,0,2927.74,43.59216,56.19952,1926.25,22.79,3.21,5.02
35997,41b1bc9d-b7ec-4092-803b-6ae2a1023605,01-12-2024,60708.53,28.86782,73.22354,1958.70,15.64,3.41,3.82,64,...,0,1,0,818.01,28.86782,73.22354,1958.70,15.64,3.41,3.82
35998,41b1bc9d-b7ec-4092-803b-6ae2a1023605,01-01-2025,61984.00,40.88148,59.33356,1284.70,26.19,2.17,5.82,64,...,0,1,0,1275.47,40.88148,59.33356,1284.70,26.19,2.17,5.82


Feature engineering: lag and rolling features added to stationary columns.

In [17]:
import pandas as pd
import numpy as np

# Load the stationary dataset
df = pd.read_csv("/Users/isha/Datathon/stationary_data1.csv")

# ✅ Only apply to stationary-transformed columns
features_to_engineer = [
    'portfolio_value_stationary',
    'monthly_contribution_stationary',
    'market_volatility_index_stationary',
    'sentiment_index_stationary',
    'macroeconomic_score_stationary'
]

# ---------- Function: Add Lag Features ----------
def add_lag_features(df, features, lags=[1, 3, 6]):
    df_lagged = df.copy()
    for feature in features:
        for lag in lags:
            df_lagged[f"{feature}_lag_{lag}"] = df_lagged.groupby("client_id")[feature].shift(lag)
    return df_lagged

# ---------- Function: Add Rolling Statistics ----------
def add_rolling_features(df, features, windows=[3, 6]):
    df_rolled = df.copy()
    for feature in features:
        for window in windows:
            df_rolled[f"{feature}_roll_mean_{window}"] = (
                df_rolled.groupby("client_id")[feature].transform(lambda x: x.rolling(window).mean())
            )
            df_rolled[f"{feature}_roll_std_{window}"] = (
                df_rolled.groupby("client_id")[feature].transform(lambda x: x.rolling(window).std())
            )
    return df_rolled

# ---------- Apply Lag + Rolling Feature Engineering ----------
df = add_lag_features(df, features=features_to_engineer, lags=[1, 3, 6])
df = add_rolling_features(df, features=features_to_engineer, windows=[3, 6])

# ---------- Drop rows with NaNs introduced by shifting/rolling ----------
df = df.dropna().reset_index(drop=True)

# ---------- Save the engineered dataset ----------
df.to_csv("/Users/isha/Datathon/engineered_stationary_data1.csv", index=False)
print("✅ Feature engineering complete: lag and rolling features added to stationary columns.")


✅ Feature engineering complete: lag and rolling features added to stationary columns.


Engineer new features: portfolio_growth_rate, contribution_ratio, eq_fi_ratio

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# --- Config ---
test_file = "/Users/isha/Datathon/stationary_data_with_smart_features1.csv"
model_path = "/Users/isha/Datathon/lstm_portfolio_forecast_model.keras"  # Update if different
output_file = "/Users/isha/Datathon/predictions.csv"
sequence_length = 24



# --- Features used in training ---
selected_features = ['employment_status_Unemployed', 'investment_goals_Wealth Accumulation', 'asset_Real Estate',
                     'risk_appetite_encoded', 'market_volatility_index_stationary_roll_mean_6',
                     'investment_goals_Home Purchase', 'eq_fi_ratio', 'savings_rate',
                     'sentiment_index_stationary_roll_mean_6', 'age', 'employment_status_Self-Employed',
                     'annual_income', 'asset_Bonds', 'macroeconomic_score_stationary_roll_mean_3',
                     'debt_to_income_ratio', 'financial_knowledge_score', 'investment_goals_Retirement',
                     'asset_Mutual Funds', 'asset_Stocks', 'net_worth', 'monthly_contribution_stationary_roll_mean_6',
                     'gender_Male', 'gender_Other', 'monthly_contribution_stationary_roll_mean_3',
                     'dependents', 'investment_horizon_years', 'employment_status_Salaried',
                     'sentiment_index_stationary_roll_mean_3', 'market_volatility_index_stationary_roll_mean_3',
                     'asset_ETFs', 'macroeconomic_score_stationary_roll_mean_6']

# --- Load test data ---
df = pd.read_csv(test_file)

# Drop rows with missing values in selected features
df = df.dropna(subset=selected_features)

# Sort by client_id and month
df = df.sort_values(['client_id', 'month'])

# Normalize features
feature_scaler = MinMaxScaler()
df[selected_features] = feature_scaler.fit_transform(df[selected_features])

# Dummy scaler for inverse transform later (we just need shape and min/max)
target_scaler = MinMaxScaler()
target_scaler.fit(np.zeros((1, 3)))  # Just for inverse_transform to work

# --- Sequence Creation ---
def create_sequences(data, features, seq_len):
    X, client_ids = [], []
    grouped = data.groupby('client_id')
    for client_id, group in grouped:
        group = group.reset_index(drop=True)
        if len(group) >= seq_len:
            for i in range(len(group) - seq_len + 1):
                seq_x = group.loc[i:i+seq_len-1, features].values.astype(np.float32)
                X.append(seq_x)
                client_ids.append(client_id)
    return np.array(X), client_ids

X_test, client_ids = create_sequences(df, selected_features, sequence_length)

# --- Load model ---
model = load_model(model_path, compile=False)


# --- Predict ---
y_pred = model.predict(X_test)

# --- Save predictions ---
pred_df = pd.DataFrame(y_pred, columns=['forecasted_value_year_1', 'forecasted_value_year_2', 'forecasted_value_year_3'])
pred_df['client_id'] = client_ids
pred_df.to_csv(output_file, index=False)

print(f"✅ Predictions saved to: {output_file}")


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
✅ Predictions saved to: /Users/isha/Datathon/predictions.csv


In [35]:
from joblib import load

target_scaler = load('/Users/isha/Datathon/target_scaler.save')  # adjust path if needed


# --- Inverse transform ---
y_pred_unscaled = target_scaler.inverse_transform(y_pred)

# --- Save unscaled predictions ---
unscaled_pred_df = pd.DataFrame(y_pred_unscaled, columns=['forecasted_value_year_1', 'forecasted_value_year_2', 'forecasted_value_year_3'])
unscaled_pred_df['client_id'] = client_ids
unscaled_pred_df.to_csv("/Users/isha/Datathon/predictions.csv", index=False)

print("📈 Unscaled predictions saved to: /Users/isha/Datathon/predictions.csv")



📈 Unscaled predictions saved to: /Users/isha/Datathon/predictions.csv
